In [1]:
import pandas as pd
import numpy as np
import requests
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.pakwheels.com/used-cars/search/-/featured_1/")

try:
    WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, ".list-unstyled.search-results"))
    )
except:
    print("Timeout waiting for element")

html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
driver.quit()

In [3]:
all_listings = soup.find_all("li", class_="classified-listing")

listing_data = []

###EXTRACT DATA FROM SOUP
def extract_data(all_listings):

    for car in all_listings:
        car_title = car.find('h3').get_text(strip=True)
        car_price = car.find("div", class_="price-details").get_text(strip=True)
        car_city = car.select_one(".list-unstyled.search-vehicle-info li").get_text(strip=True) 
        car_model_year = car.select_one(".list-unstyled.search-vehicle-info-2 li").get_text(strip=True)
        car_milage = car.select_one(".list-unstyled.search-vehicle-info-2 li:nth-of-type(2)").get_text(strip=True)
        car_fuel_type = car.select_one(".list-unstyled.search-vehicle-info-2 li:nth-of-type(3)").get_text(strip=True)
        car_power = car.select_one(".list-unstyled.search-vehicle-info-2 li:nth-of-type(4)").get_text(strip=True)
        car_transmission_type = car.select_one(".list-unstyled.search-vehicle-info-2 li:nth-of-type(5)").get_text(strip=True)
    
        listing_data.append({
            "Title": car_title,
            "Price": car_price,
            "City": car_city,
            "Model Year": car_model_year,
            "Mileage": car_milage,
            "Fuel Type": car_fuel_type,
            "Power In CC": car_power,
            "Transmission Type": car_transmission_type
        })
    
###LOAD DATA TO CSV
def load_to_csv(listing_data):
    df = pd.DataFrame(listing_data)
    df.to_csv("raw_listing_data_pakwheels.csv", index=False)
    

In [4]:
##CALLING EXTRACT DATA FUNCTION
extract_data(all_listings)
load_to_csv(listing_data)